# Text Classification Exam

Welcome to the Text Classification Practical Exam. In this exam, you will be tasked with building, training, and evaluating an NLP model to classify text data. You are provided with a labeled dataset containing both the text and its corresponding class labels.

Your objective is to develop a model that accurately predicts the class of the given text. Make sure to follow best practices in data preprocessing, model selection, and evaluation to achieve optimal results.

Good luck!
___

In [ ]:
!pip install -q --upgrade rouge-score
!pip install -q --upgrade keras-nlp

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.2/572.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 56.4 MB/s eta 0:00:00


In [ ]:
!pip install -q --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.6 MB/s eta 0:00:00


# Install and Import Needed Libraries

In [ ]:
! pip install tensorflow

In [141]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Download the Dataset

In [ ]:
!kaggle datasets download -d khaledzsa/spam-messages
!unzip spam-messages.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/spam-messages
License(s): unknown
  0% 0.00/213k [00:00<?, ?B/s]
100% 213k/213k [00:00<00:00, 81.7MB/s]
Archive:  spam-messages.zip
  inflating: spam.csv                


In [ ]:
! Unzip spam-messages.zip

/bin/bash: line 1: Unzip: command not found


In [123]:
df = pd.read_csv('/content/spam.csv')

In [ ]:
df.head()

,text,label,word_count
0,"Go until jurong point, crazy.. Available only ...",ham,20
1,Ok lar... Joking wif u oni...,ham,6
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,29
3,U dun say so early hor... U c already then say...,ham,12
4,"Nah I don't think he goes to usf, he lives aro...",ham,14


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5572 non-null   object
 1   label   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [ ]:
df['label'].value_counts()

,count
label,
ham,4825
spam,747


# Data Exploration

Before diving into preprocessing and model building, it’s important to first explore the dataset to understand its structure, distribution, and key characteristics. This step will help you gain insights into the data and guide your decisions in subsequent steps. Here’s what to consider:

1. **Inspect the Data**:
   Start by looking at the first few rows of the dataset to get a sense of its structure. Check the columns, data types, and a few sample entries. This helps to ensure that the data is loaded correctly and gives you an initial overview of the content.

2. **Check for Missing Values**:
   Identify if there are any missing values in the dataset.

3. **Distribution of Labels**:
   Examine the distribution of the target labels (classes).

4. **Text Data Characteristics (Bonus)**:
   Analyze the length of the text data. It is useful to calculate the number of words or characters in each text sample to understand how long the texts are. This will help you set a suitable `max_length` for tokenization and padding later. You can plot a histogram of text lengths to visualize the distribution.

5. **Common Words and Vocabulary (Bonus)**:
   Explore the most frequent words in the text data.

In [ ]:
# Check for Missing Values
df.isnull().sum()

,0
text,0
label,0


# Data Cleaning and Preprocessing

In this section, we will focus on cleaning and filtering the dataset, preparing it for the text classification task. We will implement the following steps:

1. **Remove missing values**:  
   First, we eliminate any rows with missing values to ensure the dataset is complete and consistent.

2. **Filter by text length (Bonus)**:  
   To maintain a uniform dataset, we will filter the text samples by a specified word count range. This ensures that the texts are neither too short to lack context nor too long to introduce unnecessary complexity.

3. **English stopwords loading**:  
   We load a list of English stopwords to filter out commonly used but contextually insignificant words. This is an important step for improving the performance of the model, as stopwords do not contribute valuable information.

4. **Text cleaning**:  
   We apply a series of text cleaning steps to standardize and simplify the text data. This involves:
   
   - **Removing links (URLs)**:  
     Any URLs present in the text are removed as they are not meaningful for classification purposes.
   
   - **Removing special characters and punctuation**:  
     This step removes any non-alphabetical characters, ensuring the text only contains meaningful words.
   
   - **Lowercasing**:  
     All text is converted to lowercase for uniformity and to avoid case sensitivity issues.
   
   - **Removing English stopwords**:  
     Words that are part of the stopwords list are removed, as they do not add value to the classification task.
   
   - **Stemming or Lemmatization**:  
     Either stemming or lemmatization is applied to reduce words to their root or base form, ensuring consistency in word forms.

5. **Final cleanup**:
   Apply the cleanup function to the feature column.

By following these steps, the text will be cleaned, filtered, and ready for tokenization!

In [ ]:
# English stopwords loading:

In [124]:
! pip install nltk

In [125]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [126]:
stop_words = set(stopwords.words('english'))

In [ ]:
print(stop_words)

{'does', 'won', "couldn't", 'aren', 'some', 'haven', 'how', 'than', 'an', 'myself', 'doing', 'now', 'ourselves', 'i', 'had', 'couldn', 'me', 'each', 'from', 'most', "that'll", 'why', 'what', 'then', "don't", "haven't", 'were', 'herself', "mightn't", 'where', 'm', 'shouldn', 'hadn', 'will', "hadn't", "weren't", 'whom', 'the', 'who', 'is', 'against', 'their', 'or', 'on', "you'll", 'they', 'she', 't', 'through', "hasn't", "doesn't", 'no', 'about', 'mightn', 'and', 'again', 'been', 'as', 'them', 'down', 'themselves', "you've", 'but', 'not', 'wasn', 'do', 'her', "should've", 'a', 'yours', 'having', 'any', 'ma', 'here', 'your', 'there', 'over', 'few', "aren't", 'don', 'too', 'yourselves', 'isn', 'shan', 'during', 'below', 'did', 's', 'o', 'such', 'himself', 'once', 'have', 'wouldn', 'has', 'with', 'doesn', 'nor', 're', "shan't", 'off', 'can', "you'd", 'which', 'into', 'up', 'out', 'more', 'at', 'needn', "needn't", 'hers', 'be', 'its', 'all', 've', 'of', 'ours', 'for', 'we', 'just', "shouldn'

In [127]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [128]:
df['clean_text'] = df['text'].apply(remove_stopwords)

In [129]:
# Filter by text length (Bonus)
df['word_count'] = df['text'].apply(lambda x: len(str(x).split()))

In [130]:
df_filtered = df[(df['word_count'] >= 5) & (df['word_count'] <= 100)]

df_filtered = df_filtered.drop(columns=['word_count'])

In [131]:
df_filtered

,text,label,clean_text
0,"Go until jurong point, crazy.. Available only ...",ham,"Go jurong point, crazy.. Available bugis n gre..."
1,Ok lar... Joking wif u oni...,ham,Ok lar... Joking wif u oni...
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,Free entry 2 wkly comp win FA Cup final tkts 2...
3,U dun say so early hor... U c already then say...,ham,U dun say early hor... U c already say... http...
4,"Nah I don't think he goes to usf, he lives aro...",ham,"Nah think goes usf, lives around though https:..."
...,...,...,...
5567,This is the 2nd time we have tried 2 contact u...,spam,2nd time tried 2 contact u. U å£750 Pound priz...
5568,Will Ì_ b going to esplanade fr home?,ham,Ì_ b going esplanade fr home?
5569,"Pity, * was in mood for that. So...any other s...",ham,"Pity, * mood that. So...any suggestions? https..."
5570,The guy did some bitching but I acted like i'd...,ham,guy bitching acted like i'd interested buying ...


In [ ]:
# Text cleaning:

In [132]:
import re

In [133]:
nltk.download('stopwords')
stemmer = ISRIStemmer()

def clean_text(text):
  text = re.sub(r'http\S+|www\S+|https\S+', '', text)
  text = re.sub(r'[^\w\s\å\Û]', '', text)
  text = re.sub(r'ـ+', '', text)
  text = " ".join([word for word in text.split()])
  text = " ".join([stemmer.stem(word) for word in text.split()])
  return text

df_filtered['clean_text'] = df_filtered['text'].apply(clean_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [119]:
df_filtered

,text,label,clean_text
0,"Go until jurong point, crazy.. Available only ...",ham,Go until jurong point crazy Available only in ...
1,Ok lar... Joking wif u oni...,ham,Ok lar Joking wif u oni
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,U dun say so early hor... U c already then say...,ham,U dun say so early hor U c already then say
4,"Nah I don't think he goes to usf, he lives aro...",ham,Nah I dont think he goes to usf he lives aroun...
...,...,...,...
5567,This is the 2nd time we have tried 2 contact u...,spam,This is the 2nd time we have tried 2 contact u...
5568,Will Ì_ b going to esplanade fr home?,ham,Will Ì_ b going to esplanade fr home
5569,"Pity, * was in mood for that. So...any other s...",ham,Pity was in mood for that Soany other suggestions
5570,The guy did some bitching but I acted like i'd...,ham,The guy did some bitching but I acted like id ...


# Tokenization, Padding, and Data Splitting

In this step, we will prepare the text data for input into a model by converting the text into numerical sequences, padding them to a uniform length, and splitting the dataset into training and testing sets. Here's an overview of the steps involved:

1. **Tokenization**:
   We use a tokenizer to convert the cleaned text into numerical sequences. You can use `Tokenizer` tokenizer from `tensorflow.keras.preprocessing.text` package or any other tokenizer you like.

2. **Text to sequences**:
   After fitting the tokenizer on the cleaned text, we transform each text into a sequence of numbers, where each number corresponds to a token (word) in the text.

3. **Padding the sequences**:
   Since different texts may vary in length, we pad the sequences to ensure they all have the same length.

4. **Label encoding**:
   The labels (target values) also need to be converted into numerical form if they are not encoded.

5. **Train-test split**:
   The dataset is divided into training and testing sets. We allocate 80% of the data for training the model and reserve 20% for testing its performance.
   
   - The **training data** consists of the padded sequences used to train the model.
   - The **training labels** are the encoded labels corresponding to the training data.
   - The **testing data** is used to assess the model’s performance after training.
   - The **testing labels** are the encoded labels corresponding to the testing data.

6. **Data shape confirmation**:
   After splitting the data, we print the shape (dimensions) of both the training and testing sets to confirm that the data is properly divided and formatted.

By the end of this step, the text data will be transformed into padded numerical sequences, the labels will be encoded, and the data will be split into training and testing sets for model development and evaluation.

In [ ]:
! pip install Tokenizer

In [134]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['clean_text'])

sequences = tokenizer.texts_to_sequences(df['clean_text'])

In [135]:
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

In [138]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(df['label'])

In [139]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, encoded_labels, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")

Training data shape: (4457, 97)


# Building the Classifier

In this step, you will design and build a NLP Classifier model to classify text data. Below is a breakdown of the key components you'll implement, but it's up to you to decide how to configure them based on your understanding and experimentation:

1. **Model Type**:
   You will use a Sequential model, which allows you to stack layers in a linear sequence.

2. **Input Layer**:
   Define the shape of the input data. Consider the dimensions of your padded sequences and set the input shape accordingly.

3. **Embedding Layer**:
   The embedding layer will convert input tokens (integers) into dense vector representations. You will need to determine the size of the input dimension (based on your vocabulary) and the output dimension (embedding size).

4. **Bidirectional Simple RNN/LSTM Layers**:
   You can add one or more recurrent layers. Consider using Bidirectional layers to capture contextual information from both directions (forward and backward). You can chose SimpleRNN/GRU/LSTM to perform this step.

5. **Dense Layers**:
   Add one or more fully connected (Dense) layers to process the output from the RNN/GRU/LSTM layers.

6. **Output Layer**:
   The output layer should match the type of classification task you're working on. Consider using appropriate activation function with appropriate number of units.

7. **Model Summary**:
   After defining your model architecture, print a summary to review the number of layers, types of layers, and total parameters.

8. **Model Compilation**:
   Finally, compile the model by selecting an optimizer, a loss function, and metrics.

In [144]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
num_classes = len(set(encoded_labels))

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 39s 223ms/step - accuracy: 0.8623 - loss: 0.3426 - val_accuracy: 0.9812 - val_loss: 0.0699
Epoch 2/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 40s 217ms/step - accuracy: 0.9913 - loss: 0.0272 - val_accuracy: 0.9812 - val_loss: 0.0609
Epoch 3/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 32s 229ms/step - accuracy: 0.9995 - loss: 0.0039 - val_accuracy: 0.9830 - val_loss: 0.0836
Epoch 4/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 38s 212ms/step - accuracy: 1.0000 - loss: 2.9056e-04 - val_accuracy: 0.9776 - val_loss: 0.0833
Epoch 5/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 42s 217ms/step - accuracy: 1.0000 - loss: 3.8519e-04 - val_accuracy: 0.9821 - val_loss: 0.1056
Epoch 6/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 46s 249ms/step - accuracy: 1.0000 - loss: 7.6788e-05 - val_accuracy: 0.9821 - val_loss: 0.1101
Epoch 7/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 30s 213ms/step - accuracy: 1.0000 - loss: 5.1625e-05 - val_accuracy: 0.9821 - val_loss: 0.1141
Epoch 8/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 41s 213ms/step - accuracy: 

In [145]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 97, 100)             │         889,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 97, 128)             │          84,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 97, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 64)                  │          41,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │              66 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,050,528 (11.64 MB)

 Trainable params: 1,016,842 (3.88 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,033,686 (7.76 MB)

# Defining Batch Size, Creating Datasets, and Training the Model

In this step, you will define the batch size, create TensorFlow Datasets for both training and testing, and train the model. The key elements to consider are outlined below, and it is up to you to choose the specific configurations based on your preferences and experimentation:

1. **Batch Size**:
   Select a batch size for training and testing. The batch size determines how many samples will be processed together in one forward and backward pass during training.

2. **Creating Datasets**:
   Use TensorFlow’s `Dataset.from_tensor_slices()` to create datasets from the training and testing data.

3. **Batching the Datasets**:
   Batch the datasets by grouping the data into batches of the specified size.

4. **Training the Model**:
   Train the model by fitting it on the training dataset for a specified number of epochs. You will also need to provide the validation data to monitor the model’s performance on unseen data during training.

5. **Tracking Training History**:
   During training, the model’s performance metrics (such as loss and accuracy) will be tracked over the epochs, and the results will be stored in the `history` object.

In [155]:
batch_size = 32

In [162]:
train_dataset = tensorflow.data.Dataset.from_tensor_slices((train_data, train_labels))
test_dataset = tensorflow.data.Dataset.from_tensor_slices((test_data, test_labels))

train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

# Model Evaluation

Once the model is trained, the next step is to evaluate its performance on the testing dataset.

1. **Evaluate the Model**:
   You will use the `evaluate()` method to assess the model’s performance on the test dataset.

2. **Testing Dataset**:
   Ensure that the testing dataset is properly prepared and batched, just like the training dataset.

4. **Loss Curve**:
   A loss curve plots the loss values for both the training and validation datasets over the epochs.

In [164]:
results = model.evaluate(test_data, test_labels)
print(f"Accuracy result: {results[1] * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 738ms/step - accuracy: 1.0000 - loss: 0.2011
Accuracy result: 100.00%


# Model Inference

In this step, you will use the trained model to make predictions on new, unseen data (inference). Here’s an outline of the key points:

1. **Create Test Sample**:
   Create a string to test your modelm the goal here is to give the model Before making predictions, ensure that the new data is preprocessed in the same way as the training data. This includes tokenization, padding, and any other transformations you applied during the data preprocessing step. The data can be single text to see the result of the prediction.

2. **Model Prediction**:
   Use the `predict()` method to feed new samples into the trained model and obtain predictions. The model will output probabilities or predicted class labels based on the type of classification task (binary or multi-class).

3. **Interpreting Predictions**:
   The model will return probabilities for each class.

In [167]:
sample_text = ['Message']
sample_sequence = tokenizer.texts_to_sequences([clean_text(sample_text[0])])

padded_sample_sequence = pad_sequences(sample_sequence, maxlen=max_length, padding='post')
predicted_label = model.predict(padded_sample_sequence)

print("Predicted label:", label_encoder.inverse_transform([predicted_label.argmax()]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Predicted label: ['ham']


# Notebook Question:
- How did you handle text preprocessing? Why did you choose this approach?

- Why did you choose this model design?

- Why did you pick this number of layers or units for the model?

- Why did you select these evaluation methods?

- Does your model show signs of overfitting or underfitting? How do you know?

- What changes could you make to improve the model and fix overfitting or underfitting?

Answer Here: